In [1]:
import glob
from rasterio.merge import merge
import rasterio
import os
import numpy as np
from fiona.crs import from_epsg
from pyproj import CRS
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask
import geopandas as gpd

In [2]:
def create_mosaic(directory, out_name):
    """Creates mosaic from all rasters (.tif format) inside given directory"""
    search_criteria = '*.tif'

    dirpath = directory
    outfp = os.path.join(dirpath, 'data', out_name)
    q = os.path.join(dirpath, search_criteria)
    
    tifs = glob.glob(q)
    # Looping through tifs
    files_to_mosaic =[]
    for fp in tifs:
        src = rasterio.open(fp)
        files_to_mosaic.append(src)
    
    # Merging to mosaic
    mosaic, out_trans = merge(files_to_mosaic)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                 "crs": CRS(src.crs) })
    with rasterio.open(outfp, "w", **out_meta) as dest:
        dest.write(mosaic)

In [3]:
def get_rice(raster_name, out_name):
    """Choosing only rice-landcover from raster and writing it as new file"""
    fp = os.path.join('Raw_Rasters/data/clipped', raster_name)
    outfp = os.path.join('Raw_Rasters/data/rice', out_name)
    
    
    data = rasterio.open(fp)
    array = data.read(1)
    transform, width, height = calculate_default_transform(
        data.crs, 4326, data.width, data.height, *data.bounds)
    
    array = array == 11
    array = array.astype(int)
    with rasterio.open(
        outfp,
        mode="w",
        driver="GTiff",
        height=height,
        width=width,
        count=1,
        dtype=array.dtype,
        crs=data.crs,
        transform=transform
    ) as new_dataset:
        new_dataset.write(array, 1)

In [4]:
def clip_raster(raster, coords, out_name):
    """Clipping the raster based on the coordinates given from a bounding box. Bounding box needs to be calculated beforehand"""
    dirpath = 'Raw_Rasters/data/'
    fp = os.path.join(dirpath, raster)
    outfp = os.path.join(dirpath, 'clipped', out_name)
    data = rasterio.open(fp)
    # Cropping by bounding box
    out_meta = data.meta.copy()
    out_raster, out_trans = mask(dataset=data, shapes=coords, crop=True)
    #if crs is none, setting crs to epsg:4326
    if data.crs == None:
        out_crs = {'init' : 'epsg:4326'}
    else:
        out_crs = CRS(data.crs)
    # updating meta
    out_meta.update({"driver": "GTiff",
                 "height": out_raster.shape[1],
                 "width": out_raster.shape[2],
                 "transform": out_trans,
                 "crs": out_crs 
                 }
                )
    with rasterio.open(outfp, "w", **out_meta) as dest:
        dest.write(out_raster)
        

In [5]:
def align_to_raster(raster_to, raster_from, out_name):
    """Aligning rasters by getting the transformation parameters from landcover raster and band values from 
    irrigation raster. The base code from: https://rasterio.readthedocs.io/en/latest/topics/reproject.html."""
    
    dirpath = 'Raw_Rasters/data'
    fp1 = os.path.join(dirpath,'clipped', raster_to)
    fp2 = os.path.join(dirpath,'clipped', raster_from)
    outfp = os.path.join(dirpath, out_name)
    dst_crs = 4326
    with rasterio.open(fp1) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
             'width': width,
            'height': height
    })
    # Writing to 'Rasters/irrigation_aligned' with irrigation bands as source. Source transformation from Lao_Cam mosaic. 
    with rasterio.open(outfp, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(rasterio.open(fp2), i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)